# Task 2: Quantitative Analysis using TA-Lib

## Objective
Perform technical analysis on stock price data using TA-Lib to calculate:
- Moving Averages (SMA, EMA)
- RSI (Relative Strength Index)
- MACD (Moving Average Convergence Divergence)
- Bollinger Bands
- Other technical indicators

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib as ta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## Load and Prepare Data

In [ ]:
# Load stock data
tickers = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
stock_data = {}

for ticker in tickers:
    df = pd.read_csv(f'../data/Data/{ticker}.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df.set_index('Date', inplace=True)
    stock_data[ticker] = df
    print(f"{ticker}: {len(df)} records")

stock_data['AAPL'].head()

## Calculate Technical Indicators

In [ ]:
# Calculate indicators for all stocks
for ticker in tickers:
    df = stock_data[ticker]
    
    # Moving Averages
    df['SMA_20'] = ta.SMA(df['Close'], timeperiod=20)
    df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
    df['SMA_200'] = ta.SMA(df['Close'], timeperiod=200)
    df['EMA_12'] = ta.EMA(df['Close'], timeperiod=12)
    df['EMA_26'] = ta.EMA(df['Close'], timeperiod=26)
    
    # RSI
    df['RSI'] = ta.RSI(df['Close'], timeperiod=14)
    
    # MACD
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
    # Bollinger Bands
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = ta.BBANDS(df['Close'], timeperiod=20)
    
    # ATR (Volatility)
    df['ATR'] = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Stochastic
    df['STOCH_k'], df['STOCH_d'] = ta.STOCH(df['High'], df['Low'], df['Close'])
    
    # OBV
    df['OBV'] = ta.OBV(df['Close'], df['Volume'])
    
    # Returns
    df['Daily_Return'] = df['Close'].pct_change()
    df['Cumulative_Return'] = (1 + df['Daily_Return']).cumprod() - 1
    
    stock_data[ticker] = df

print("Technical indicators calculated")
stock_data['AAPL'][['Close', 'SMA_20', 'RSI', 'MACD']].tail()

## Visualizations

In [ ]:
# Price and Moving Averages
fig, axes = plt.subplots(3, 2, figsize=(18, 12))
axes = axes.flatten()

for idx, ticker in enumerate(tickers):
    df = stock_data[ticker].tail(252)
    ax = axes[idx]
    ax.plot(df.index, df['Close'], label='Close', linewidth=2)
    ax.plot(df.index, df['SMA_20'], label='SMA 20', alpha=0.7)
    ax.plot(df.index, df['SMA_50'], label='SMA 50', alpha=0.7)
    ax.set_title(f'{ticker} - Price & Moving Averages')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# RSI
fig, axes = plt.subplots(3, 2, figsize=(18, 12))
axes = axes.flatten()

for idx, ticker in enumerate(tickers):
    df = stock_data[ticker].tail(252)
    ax = axes[idx]
    ax.plot(df.index, df['RSI'], color='purple', linewidth=2)
    ax.axhline(70, color='r', linestyle='--', alpha=0.7)
    ax.axhline(30, color='g', linestyle='--', alpha=0.7)
    ax.fill_between(df.index, 30, 70, alpha=0.1)
    ax.set_title(f'{ticker} - RSI')
    ax.set_ylim(0, 100)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# MACD
fig, axes = plt.subplots(3, 2, figsize=(18, 12))
axes = axes.flatten()

for idx, ticker in enumerate(tickers):
    df = stock_data[ticker].tail(252)
    ax = axes[idx]
    ax.plot(df.index, df['MACD'], label='MACD', linewidth=2)
    ax.plot(df.index, df['MACD_signal'], label='Signal', linewidth=2)
    ax.bar(df.index, df['MACD_hist'], label='Histogram', alpha=0.3)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.set_title(f'{ticker} - MACD')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cumulative Returns Comparison
plt.figure(figsize=(16, 8))
for ticker in tickers:
    df = stock_data[ticker]
    plt.plot(df.index, df['Cumulative_Return'] * 100, label=ticker, linewidth=2)

plt.title('Cumulative Returns Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative Return (%)')
plt.legend()
plt.grid(alpha=0.3)
plt.axhline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

## Summary Statistics

In [ ]:
# Calculate summary statistics
summary = {}
for ticker in tickers:
    df = stock_data[ticker]
    summary[ticker] = {
        'Current_Price': df['Close'].iloc[-1],
        'Mean_Return': df['Daily_Return'].mean() * 100,
        'Volatility': df['Daily_Return'].std() * np.sqrt(252) * 100,
        'Sharpe_Ratio': (df['Daily_Return'].mean() * 252) / (df['Daily_Return'].std() * np.sqrt(252)),
        'Total_Return': df['Cumulative_Return'].iloc[-1] * 100,
        'Current_RSI': df['RSI'].iloc[-1]
    }

summary_df = pd.DataFrame(summary).T
print("\nSummary Statistics:")
summary_df